In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model = MLP()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

transform = transforms.Compose([
    transforms.ToTensor()
]) # maybe play with normalization here


train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_dataset, val_dataset = random_split(train_dataset, [55000, 5000])
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

def validate(model, val_loader):
    model.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in val_loader:
            output = model(data)
            val_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    val_loss /= len(val_loader.dataset)
    accuracy = 100. * correct / len(val_loader.dataset)
    print(f'Validation set: Average loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%')

# Training loop
def train(epoch, log_interval=200):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            pass
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

num_epochs = 5

for epoch in range(1, num_epochs + 1):
    train(epoch)
    validate(model, val_loader)

validate(model, test_loader)

Train Epoch: 1 [0/55000 (0%)]	Loss: 2.302529
Train Epoch: 1 [6400/55000 (12%)]	Loss: 0.245718
Train Epoch: 1 [12800/55000 (23%)]	Loss: 0.445461
Train Epoch: 1 [19200/55000 (35%)]	Loss: 0.195971
Train Epoch: 1 [25600/55000 (47%)]	Loss: 0.398889
Train Epoch: 1 [32000/55000 (58%)]	Loss: 0.194513
Train Epoch: 1 [38400/55000 (70%)]	Loss: 0.117087
Train Epoch: 1 [44800/55000 (81%)]	Loss: 0.088551
Train Epoch: 1 [51200/55000 (93%)]	Loss: 0.241987
Validation set: Average loss: 0.0048, Accuracy: 95.62%
Train Epoch: 2 [0/55000 (0%)]	Loss: 0.252680
Train Epoch: 2 [6400/55000 (12%)]	Loss: 0.037956
Train Epoch: 2 [12800/55000 (23%)]	Loss: 0.179405
Train Epoch: 2 [19200/55000 (35%)]	Loss: 0.060457
Train Epoch: 2 [25600/55000 (47%)]	Loss: 0.252910
Train Epoch: 2 [32000/55000 (58%)]	Loss: 0.070517
Train Epoch: 2 [38400/55000 (70%)]	Loss: 0.303405
Train Epoch: 2 [44800/55000 (81%)]	Loss: 0.093354
Train Epoch: 2 [51200/55000 (93%)]	Loss: 0.017407
Validation set: Average loss: 0.0035, Accuracy: 96.78%
Tr